In [1]:
import numpy as np
from helper import softmax, sigmoid

In [2]:
def rnn_cell_forward(x_t, a_prev_t, parameters):
    """
    This implements the rnn block of the rnn-network for forward propagation.
    Suitable for many-to-many architecture (Tx=Ty).
    Arguments:
        x_t:Input data for the current timestep, having dimensions (n_x, m)
        a_prev_t: Hidden state for the previous time-step , having dimensions of (n_a, m)
        Parameters is a dictionary of parameters: Waa, Wax, Wya, by, ba
    
    Returns:
        a_next_t:Hidden state for the current timestep , having dimensions of (n_a, m)
        y_pred_t: Prediction for the current time-step , having dimensions (n_y, m)
    
    """
    Wax=parameters["Wax"]
    Waa=parameters["Waa"]
    Wya=parameters["Wya"]
    by=parameters["by"]
    ba=parameters["ba"]
    
    a_next_t=np.tanh(np.dot(Wax, x_t)+np.dot(Waa, a_prev_t)+ba)
    y_pred_t=softmax(np.dot(Wya, a_next_t)+by)
    return (a_next_t, y_pred_t)

In [3]:
def rnn_forward_pass(x,a0, parameters):
    """
    This implements the forward pass of the rnn network with RNN block.
    Arguments:
        x: Input data, having dimensions of (n_x, m, T_x)
        a0:Initial hidden state , having dimensions of (n_a, m)
        Parameters is a dictionary of parameters: Waa, Wax, Wya, by, ba
        
    Returns :
        a:Hidden state for all the time-steps, having dimensions of (n_a, m, T_x)
        y_pred: Predictions for all the time-steps, having dimensions of (n_y, m, T_y) . Here, T_x=T_y
    
    """
    Wya=parameters["Wya"]
    
    n_x, m, T_x=x.shape
    n_y, n_a=Wya.shape
    
    a=np.zeros((n_a, m, T_x))
    y_pred=np.zeros((n_y, m, T_x))
    a_next=a0
    
    for i in range(T_x):
        a[:,:,i],y_pred[:,:,i]=rnn_cell_forward(x[:,:,i], a_next, parameters)
        a_next=a[:,:,i]
    return (a, y_pred)

In [4]:
def lstm_cell_forward(x_t, a_prev_t, c_prev_t, parameters):
    """
    This implements the LSTM block of the rnn-network for forward propagation.
    Arguments:
        x_t: Input data for the current time-step, having dimensions of (n_x, m)
        a_prev_t: Hidden state for the previous time-step, having dimensions of (n_a, m)
        c_prev_t: Cell state for the previous time-step, having dimensions of (n_a, m)
        Parameters is a dictionary of parameters: Wy, Wc, Wu, Wf, Wo, by, bc, bu, bf, bo
    
    Returns:
        a_next_t: Hidden state for the current time-step, having dimensions of (n_a, m)
        c_next_t: Cell state for the current time-step, having dimensions of (n_a, m)
        y_pred_t: Prediction for thr current time-step, having dimensions of (n_y, m)
        
    """
    
    Wy=parameters["Wy"]
    by=parameters["by"]
    
    Wc=parameters["Wc"]
    bc=parameters["bc"]
    
    Wu=parameters["Wu"]
    bu=parameters["bu"]
    
    Wf=parameters["Wf"]
    bf=parameters["bf"]
    
    Wo=parameters["Wo"]
    bo=parameters["bo"]
    
    concat=np.concatenate((a_prev_t, x_t), axis=0)
    
    c_tilda_t=np.tanh(np.dot(Wc, concat)+bc)
    ug=sigmoid(np.dot(Wu, concat)+bu)
    fg=sigmoid(np.dot(Wf, concat)+bf)
    og=sigmoid(np.dot(Wo, concat)+bo)
    
    c_next_t=ug*c_tilda_t+fg*c_prev_t
    a_next_t=og*(np.tanh(c_next_t))
    y_pred_t=softmax(np.dot(Wy, a_next_t)+by)
    
    return (a_next_t,c_next_t, y_pred_t)

In [5]:
def lstm_forward_pass(x, a0, parameters):
    """
    This implements the forward pass of rnn-network with LSTM block.
    Arguments:
        x: Input data, having dimensions of (n_x, m, T_x)
        a0:Initial hidden state , having dimensions of (n_a, m)
        Parameters is a dictionary of parameters: Wy, Wc, Wu, Wf, Wo, by, bc, bu, bf, bo
    
    Returns:
        a:Hidden state for all the time-steps, having dimensions of (n_a, m, T_x)
        y_pred: Predictions for all the time-steps, having dimensions of (n_y, m, T_y) . Here, T_x=T_y
        c:Cell state for all the time-steps, having dimensions of (n_a,m, T_x)
        
    """
    
    n_x, m, T_x=x.shape
    ny, na=parameters["Wy"].shape
    
    a_next_t=a0
    c_next_t=np.zeros((na, m))
    a=np.zeros((na, m, T_x))
    c=np.zeros((na, m, T_x))
    y_pred=np.zeros((ny, m, T_x))
    
    for i in range(T_x):
        a[:,:,i], c[:, :, i], y_pred[:,:,i]=lstm_cell_forward(x[:,:,i], a_next_t, c_next_t, parameters)
        a_next_t=a[:,:,i]
        c_next_t=c[:,:,i]
    return (a,y_pred,c)

In [6]:
def gru_cell_forward(x_t, a_prev_t, parameters):
    """
    This implements the GRU block of the rnn-network for forward propagation.
    Arguments:
        x_t: Input data for the current time-step, having dimensions of (n_x, m)
        a_prev_t: Hidden state for the previous time-step, having dimensions of (n_a, m)
        Parameters is a dictionary of parameters: Wy, Wc, Wu, Wr, by, bc, bu, br
    
    Returns:
        a_next_t: Hidden state for the current time-step, having dimensions of (n_a, m)
        y_pred_t: Prediction for thr current time-step, having dimensions of (n_y, m)
    
    """
    
    Wu=parameters["Wu"]
    bu=parameters["bu"]
    
    Wr=parameters["Wr"]
    br=parameters["br"]
        
    Wc=parameters["Wc"]
    bc=parameters["bc"]
    
    Wy=parameters["Wy"]
    by=parameters["by"]
    
    concat=np.concatenate((a_prev_t, x_t), axis=0)
    
    update_gate=sigmoid(np.dot(Wu, concat)+bu)
    relevance_gate=sigmoid(np.dot(Wr, concat)+br)
    
    concat_for_c_tilda=np.concatenate((update_gate*a_prev_t, x_t), axis=0)
    
    c_tilda_t=np.tanh(np.dot(Wc, concat_for_c_tilda)+bc)
    a_next_t=update_gate*c_tilda_t+(1-update_gate)*a_prev_t
    y_pred_t=softmax(np.dot(Wy, a_next_t)+by)
    
    return (a_next_t, y_pred_t)

In [9]:
def gru_forward_pass(x, a0, parameters):
    """
    This implements the forward pass of rnn-network with GRU block.
    Arguments:
        x: Input data, having dimensions of (n_x, m, T_x)
        a0:Initial hidden state , having dimensions of (n_a, m)
        Parameters is a dictionary of parameters: Wy, Wc, Wu, Wr, by, bc, bu, br
    
    Returns:
        a:Hidden state for all the time-steps, having dimensions of (n_a, m, T_x)
        y_pred: Predictions for all the time-steps, having dimensions of (n_y, m, T_y) . Here, T_x=T_y

    """
    nx, m, T_x=x.shape
    a_next=a0
    n_y, n_a=parameters["Wy"].shape
    a=np.zeros((n_a, m, T_x))
    y_pred=np.zeros((n_y, m, T_x))
    for i in range(T_x):
        a[:,:,i], y_pred[:,:,i]=gru_cell_forward(x[:,:,i], a_next, parameters)
        a_next=a[:,:,i]
    return (a, y_pred)